# Previsione della Qualità dell'Aria - **Preparazione dei Dati**

**Progetto di Data Intensive**  
**Autore:** Martin Tomassi, Jacopo Vasi  
**Email:** martin.tomassi@studio.unibo.it , jacopo.vasi@studio.unibo.it  
**Corso:** Data Intensive, Università di Bologna  
**Data:** Aprile 2025

## Caricamento dei Datasets ed Import Librerie



In [1]:
import os
import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

try:
    import google.colab
    running_in_colab = True
except ImportError:
    running_in_colab = False

from sklearn.ensemble import IsolationForest

from IPython.display import clear_output

N_JOBS = -1
RANDOM_STATE = 42

if running_in_colab:
    print("Running on Google Colab")
    !apt-get update -qq
    !apt-get install -qq git-lfs
    !git lfs install
    !git clone https://github.com/vMxster/Data_Project.git
    !cd Data_Project && git lfs pull
    zip_india_path   = "Data_Project/Datasets/dataset_india.zip"
    zip_china_path   = "Data_Project/Datasets/dataset_china.zip"
else:
    print("Running locally in Jupyter")
    zip_india_path   = "Datasets/dataset_india.zip"
    zip_china_path   = "Datasets/dataset_china.zip"



# India Dataset

india_extract_to = "datasets/india"
os.makedirs(india_extract_to, exist_ok=True)

with zipfile.ZipFile(zip_india_path, 'r') as z:
    z.extractall(india_extract_to)

print("\nEstratti:\n")
for root, _, files in os.walk(india_extract_to):
    for f in files:
        print(os.path.join(root, f))

# Cina Dataset

china_extract_to = "datasets/china"
os.makedirs(china_extract_to, exist_ok=True)

with zipfile.ZipFile(zip_china_path, 'r') as z:
    z.extractall(china_extract_to)

print("\nEstratti:\n")
for root, _, files in os.walk(china_extract_to):
    for f in files:
        print(os.path.join(root, f))

Running on Google Colab
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Git LFS initialized.
fatal: destination path 'Data_Project' already exists and is not an empty directory.

Estratti:

datasets/india/CG004.csv
datasets/india/GJ015.csv
datasets/india/RJ012.csv
datasets/india/WB014.csv
datasets/india/CG008.csv
datasets/india/UK001.csv
datasets/india/TG008.csv
datasets/india/DL018.csv
datasets/india/OR002.csv
datasets/india/AP006.csv
datasets/india/MH003.csv
datasets/india/WB008.csv
datasets/india/MH005.csv
datasets/india/DL001.csv
datasets/india/RJ011.csv
datasets/india/MH010.csv
datasets/india/MH032.csv
datasets/india/HR021.csv
datasets/india/RJ004.csv
datasets/india/WB003.csv
datasets/india/AS009.csv
datasets/india/HR001.csv
datasets/india/CG002.csv
datasets/india/DL032.csv
datasets/india/KA029.csv
datasets/india/RJ008.csv
datasets/india/MN00

# Dataset sulla **Qualità dell'Aria in India**



Il dataset in questione è stato messo a disposizione dal Central Pollution Control Board (CPCB), l’ente ufficiale del Governo indiano deputato al monitoraggio e alla gestione dell’inquinamento atmosferico, al fine di raccogliere informazioni relative alle condizioni della qualità dell’aria in 453 città indiane nel periodo compreso tra il 2010 e il 2023.

Sempre citando la documentazione ufficiale, il dataset permette di indagare su numerose variabili ambientali e parametri atmosferici che includono:

- PM10 e PM2.5: concentrazioni di particolato in ug/m³;
- CO e CO₂: rispettivamente monossido e anidride carbonica, misurati in vari formati (mg/m³, ppm, ecc.);
- NO, NO₂ e NOx: varianti degli ossidi di azoto, riportati in unità adatte (ug/m³, ppb, ppm);
- SO₂, NH₃ e altri inquinanti quali Benzene, CH₄, e composti organici come MP-Xylene, Eth-Benzene, O Xylene, e Xylene;
- Parametri meteorologici e ambientali quali temperatura, pressione barometrica, umidità relativa, velocità e direzione del vento, radiazione solare e precipitazioni.

In aggiunta al dataset principale, è disponibile anche il file “stations_info.csv”, che rappresenta una guida di riferimento per approfondire le informazioni relative alle diverse stazioni di monitoraggio. Questo file include le seguenti intestazioni:

- file_name: nome del file associato alla stazione;
- state: lo stato in cui è ubicata la stazione;
- city: la città in cui la stazione opera;
- agency: l’ente responsabile della gestione della stazione;
- station_location: dettagli aggiuntivi riguardanti la posizione;
- start_month, start_month_num e start_year: informazioni sulla data di inizio della raccolta dati per ciascuna stazione.

Si procede alla lettura del file contenente le informazioni relative alle varie stazioni di monitoraggio. Successivamente, verranno rimosse alcune colonne ritenute non necessarie per l'analisi in corso, al fine di semplificare la struttura del dataset ed enfatizzare solo le informazioni rilevanti.

In [2]:
df_states = pd.read_csv(f'{india_extract_to}/stations_info.csv')
df_states.drop(columns=['agency', 'station_location', 'start_month'], inplace=True)
df_states.head()

,file_name,state,city,start_month_num,start_year
0,AP001,Andhra Pradesh,Tirupati,7,2016
1,AP002,Andhra Pradesh,Vijayawada,5,2017
2,AP003,Andhra Pradesh,Visakhapatnam,7,2017
3,AP004,Andhra Pradesh,Rajamahendravaram,9,2017
4,AP005,Andhra Pradesh,Amaravati,11,2017


Si crea una lista di tutti gli stati presenti nel dataset, assicurandosi di includere ciascun nome una sola volta. Questo passaggio fornisce una visione d’insieme delle regioni coperte dai dati, supportando analisi geografiche e suddivisioni successive.

In [3]:
unique_states = df_states['state'].unique()
unique_states

array(['Andhra Pradesh', 'Arunachal Pradesh', 'Assam', 'Bihar',
       'Chhattisgarh', 'Chandigarh', 'Delhi', 'Gujarat',
       'Himachal Pradesh', 'Haryana', 'Jharkhand', 'Jammu and Kashmir',
       'Karnataka', 'Kerala', 'Maharashtra', 'Meghalaya', 'Manipur',
       'Madhya Pradesh', 'Mizoram', 'Nagaland', 'Odisha', 'Punjab',
       'Puducherry', 'Rajasthan', 'Sikkim', 'Telangana', 'Tamil Nadu',
       'Tripura', 'Uttarakhand', 'Uttar Pradesh', 'West Bengal'],
      dtype=object)

Questa funzione consente di raccogliere, integrare e unificare dati relativi alla qualità dell’aria provenienti da più file CSV associati a diverse località di un determinato stato. La strategia prevede innanzitutto l’individuazione automatica di tutti i file relativi ad uno specifico stato, grazie all’utilizzo di un codice identificativo presente all’inizio del nome di ciascun file. Una volta selezionati i file pertinenti, il contenuto di ciascuno viene letto e trasformato in una struttura dati standardizzata. Durante questo processo, viene aggiunta ad ogni insieme di dati un’informazione supplementare che indica la città di appartenenza, garantendo così che ogni record contenga il riferimento geografico completo. Alla fine, i singoli dataset vengono combinati in un’unica struttura dati, offrendo un quadro complessivo che facilita un’analisi omogenea e dettagliata delle tendenze e delle variabili relative alla qualità dell’aria all’interno dello stato esaminato.

In [4]:
from pathlib import Path
import pandas as pd
import glob

def combine_state_df(state_name):
    state_code = df_states[df_states['state'] == state_name]['file_name'].iloc[0][:2]

    if running_in_colab:
        state_files = glob.glob(f'{india_extract_to}/{state_code}*.csv')
    else:
        state_files = [str(p.as_posix()) for p in Path(india_extract_to).glob(f'{state_code}*.csv')]

    combined_df = []
    for state_file in state_files:
        file_name = state_file.split(f'{india_extract_to}/')[1][:-4]  # remove .csv
        file_df = pd.read_csv(state_file)

        # Add city and state information
        file_df['city'] = df_states[df_states['file_name'] == file_name]['city'].values[0]
        file_df['city'] = file_df['city'].astype('string')
        file_df['state'] = state_name  # Add the state name

        combined_df.append(file_df)

    return pd.concat(combined_df, ignore_index=True)

In [ ]:
# Get list of all unique state names
unique_states = df_states['state'].unique()

# Combine all states' data
df_india = pd.concat([combine_state_df(state) for state in unique_states], ignore_index=True)

# Optional: show all columns when displaying the DataFrame
pd.set_option('display.max_columns', None)

# Inspect the combined DataFrame
df_india.info()
df_india.head()

Dall’analisi del dataframe risulta che per lo stato di Delhi sono presenti 58 metriche diverse distribuite su un totale di 2.796.171 record.

## Pre-elaborazione dei dati

#### Utilizzo di ‘From Date’ come indice temporale

Nel dataset sono presenti due colonne di tipo oggetto, `From Date` e `To Date`, che indicano l’inizio e la fine di ciascuna finestra oraria di misurazione. Per gestire efficacemente le serie storiche, trasformiamo `From Date` in un indice datetime, eliminando poi la colonna `To Date`.

In [ ]:
def create_dt_index(dataframe):
    dataframe = dataframe.drop(columns='To Date')
    dataframe['From Date'] = pd.to_datetime(dataframe['From Date'])
    dataframe = dataframe.rename(columns={'From Date': 'datetime'})
    return dataframe.set_index('datetime')

In [ ]:
df_india = create_dt_index(df_india)
df_india.head(2)

### Feature Reduction

Dall’esame del dataframe emerge che alcune colonne contengono informazioni sovrapposte. Per individuare possibili duplicazioni e fusioni, confronteremo gli andamenti delle medie annuali delle variabili raggruppate. Per ciascun gruppo, aggregheremo i valori per anno e tracceremo un grafico a linee in una griglia, in modo da mettere in evidenza trend comuni e facilitare l’individuazione di correlazioni tra le feature.

In [ ]:
def plot_feature_similarities(dataframe, feature_groups, columns=2):
    rows = int((len(feature_groups)/columns)//1)
    fig, axes = plt.subplots(rows, columns, figsize=(13, 4*rows))
    fig.tight_layout(pad=3.0)

    row_num = 0
    col_num = 0
    for pos, group in enumerate(feature_groups):
        if pos % columns == 0 and pos != 0:
            row_num += 1
            col_num = 0

        for feature in feature_groups[group]:
            df_feature = dataframe[dataframe[feature].notnull()][feature]
            df_feature = df_feature.groupby([df_feature.index.year]).mean(numeric_only=True)
            sns.lineplot(data=df_feature, label=feature, ax=axes[row_num, col_num])
        axes[row_num, col_num].set_title(group)
        axes[row_num, col_num].set(xlabel=None)
        col_num += 1

    plt.plot()

In [ ]:
groups = {
    'Xylene':            ['Xylene (ug/m3)', 'Xylene ()'],
    "MP-Xylene":         ['MP-Xylene (ug/m3)', 'MP-Xylene ()'],
    'Wind Direction':   ["WD (degree)", "WD (degree C)", "WD (deg)", "WD ()"],
    'Ozone':             ['Ozone (ug/m3)', 'Ozone (ppb)'],
    'Nitrogen Oxides':   ['NOx (ug/m3)', 'NOx (ppb)'],
    'Relative humidity':  ['RH (%)', 'RH ()'],
    'Solar Radiation': ['SR (W/mt2)', 'SR ()'],
    'Air Temperature':  ['AT (degree C)', 'AT ()']
}

plot_feature_similarities(df_india, groups, columns=2)

Procediamo quindi con l’analisi descrittiva delle variabili raggruppate. Dopo aver raccolto in un’unica lista tutte le feature interessate, calcolo per ciascuna statistica di base — media, deviazione standard, valori minimo e massimo — formattando i risultati con tre decimali per migliorarne la leggibilità. Questo step permette di valutare rapidamente la scala e la distribuzione delle variabili, facilitando le decisioni su eventuali fusioni o eliminazioni di feature.


In [ ]:
all_groups = [item for sublist in list(groups.values()) for item in sublist]
df_india[all_groups].describe().map(lambda x: f"{x:0.3f}")

Dopo aver esaminato in dettaglio la tabella, avvio la riduzione delle colonne duplicate aggregando quelle che rappresentano la stessa variabile con nomi diversi. Per farlo definisco un dizionario in cui ogni chiave è il nome unificato della variabile e i valori sono le etichette alternative. La funzione itera sul dizionario, trasferendo i valori non nulli dalle colonne secondarie a quella principale e cancellando infine le colonne ridondanti. Questo passaggio semplifica il dataset, eliminando le duplicazioni e facilitando le analisi future.

In [ ]:
reduction_groups = {
    "Xylene (ug/m3)":    ["Xylene ()"],
    "MP-Xylene (ug/m3)": ["MP-Xylene ()"],
    "Benzene (ug/m3)":   ["Benzene ()"],
    "Toluene (ug/m3)":   ["Toluene ()"],
    "SO2 (ug/m3)":       ["SO2 ()"],
    "NOx (ug/m3)":       ["NOx (ppb)"],
    "Ozone (ug/m3)":     ["Ozone (ppb)"],
    "AT (degree C)":     ["AT ()"],
    "WD (degree)":       ["WD (degree C)", "WD (deg)", "WD ()"],
    "WS (m/s)":          ["WS ()"]
}

In [ ]:
def merge_columns(dataframe, columns):
    for column, cols_to_merge in columns.items():
        if column not in dataframe.columns and any(name in dataframe.columns for name in cols_to_merge):
            dataframe[column] = np.nan

        for col_name in cols_to_merge:
            if col_name in dataframe.columns:
                dataframe[column] = dataframe[column].fillna(dataframe[col_name])
                dataframe = dataframe.drop(columns=[col_name])

    return dataframe

In [ ]:
df_india = merge_columns(df_india, reduction_groups)

### Verifica dei valori mancanti

Il primo passo consiste nel quantificare quanti dati mancanti siano presenti per ciascuna delle feature selezionate.

In [ ]:
df_india.isnull().sum().sort_values(ascending=False)

In [ ]:
df_india = df_india.dropna(how='all')
df_india = df_india.dropna(how='all', axis='columns')

In [ ]:
def get_null_info(dataframe):
    null_vals = dataframe.isnull().sum()

    df_null_vals = pd.concat({'Null Count': null_vals,
                              'Percentage of Missing Values (%)': round(null_vals * 100 / len(dataframe), 2)}, axis=1)

    return df_null_vals.sort_values(by=['Null Count'], ascending=False)

In [ ]:
df_india_null_info = get_null_info(df_india)

plt.figure(figsize=(8, 10))
sns.barplot(data=df_india_null_info, x='Percentage of Missing Values (%)', y=df_india_null_info.index, orient='h', color='steelblue')
plt.show()
df_india.head()

### Informazioni sul numero dei valori mancanti del dataset

Finora abbiamo analizzato solo un singolo stato. Potremmo avere una migliore percezione dei dati mancanti se analizzassimo l'intero Dataset.

In [ ]:
df_india.head()

In [ ]:
def get_overall_ds_info():
    features = {}
    total_records = 0

    for i, state_name in enumerate(unique_states):
        clear_output(wait=False)

        temp_df = combine_state_df(state_name)
        temp_df = create_dt_index(temp_df)
        temp_df = temp_df.dropna(how='all')

        comparisons = get_null_info(temp_df)

        total_records += df_india.shape[0]

        for feature in comparisons.index:
            if feature in features:
                features[feature] += comparisons.loc[[feature]]['Null Count'].values[0]
            else:
                features[feature] = comparisons.loc[[feature]]['Null Count'].values[0]

    ds_null_info = pd.DataFrame.from_dict(features, orient='index', columns=['Null Count'])
    ds_null_info['Percentage of Missing Values (%)'] = round(ds_null_info['Null Count'] * 100 / total_records, 2)
    ds_null_info = ds_null_info.sort_values(by=['Null Count'], ascending=False)
    return ds_null_info

In [ ]:
overall_ds_info = get_overall_ds_info()

plt.figure(figsize=(8, 16))
sns.barplot(data=overall_ds_info, x='Percentage of Missing Values (%)', y=overall_ds_info.index, orient='h', color='steelblue')
plt.show()

### Eliminare i valori mancanti per soglia

Tornando al dataframe della capitale Delhi, possiamo eliminare le colonne che contengono una certa soglia (cioè > 40%) di valori mancanti.

In [ ]:
threshold = 0.6
df_india = df_india.dropna(thresh=df_india.shape[0]*threshold, axis=1)

In [ ]:
get_null_info(df_india)

### Analisi esplorativa dei dati

Sto raccogliendo le metriche iniziali in diversi gruppi. Ciò consentirà di effettuare confronti migliori.

In [ ]:
pollutants = {
    'Particulate Matter' : ['PM2.5 (ug/m3)', 'PM10 (ug/m3)'],
    'Nitrogen Compounds' : ['NOx (ug/m3)', 'NO (ug/m3)', 'NO2 (ug/m3)', 'NH3 (ug/m3)'],
    'Hydrocarbons' : ['Benzene (ug/m3)', 'Eth-Benzene (ug/m3)', 'Xylene (ug/m3)', 'MP-Xylene (ug/m3)', 'O Xylene (ug/m3)', 'Toluene (ug/m3)'],
    'Carbon Monoxide': ['CO (mg/m3)'],
    'Sulfur Dioxide': ['SO2 (ug/m3)'],
    'Ozone Concentration' : ['Ozone (ug/m3)']
}

other_metrics = {
    'Solar Radiation' : ['SR (W/mt2)'],
    'Temperatures' : ['Temp (degree C)', 'AT (degree C)'],
    'Relative Humidity' : ['RH (%)'],
    'Rainfall' : ['RF (mm)'],
    'Barometric Pressure' : ['BP (mmHg)'],
    'Wind Direction' : ['WD (degree)'],
    'Wind Speed' : ['WS (m/s)']
}

### Frequenze temporali

Cominciamo a raggruppare il nostro DataFrame per varie frequenze temporali.

In [ ]:
slice_groups = {
    'Group by Day':   df_india.groupby(pd.Grouper(freq='1D')).mean(numeric_only=True),
    'Group by Month': df_india.groupby(pd.Grouper(freq='1ME')).mean(numeric_only=True),
    'Group by Year':  df_india.groupby(pd.Grouper(freq='1YE')).mean(numeric_only=True)
}

In [ ]:
def plot_features_by_group(features, slice_groups):
    for feature in features:
        fig, ax = plt.subplots(1, 1, figsize=(12, 4))
        fig.suptitle(feature)

        labels = []
        for i, (group, group_df) in enumerate(slice_groups.items()):
            data_slice = group_df[group_df.columns.intersection(pollutants[feature])]

            if feature == "Nitrogen Compounds":
                data_slice = data_slice.drop(['NO (ug/m3)', 'NO2 (ug/m3)'], axis=1)

            data_slice.plot(kind="line", ax=ax)

            for column in data_slice.columns:
                labels.append(f'{column} [{group}]')

        ax.set(xlabel=None)
        ax.legend(labels)
        plt.plot()

In [ ]:
features_to_plot = ['Particulate Matter', 'Carbon Monoxide', 'Ozone Concentration', 'Nitrogen Compounds']
plot_features_by_group(features_to_plot, slice_groups)
df_india.head()

### Analisi stagionale su base annua
Dalle metriche selezionate emergono potenziali pattern di tipo stagionale. Per approfondire questa osservazione, eseguiamo un’analisi dettagliata delle variazioni stagionali nell’arco di un anno. Come punto di partenza, prenderemo in considerazione un sottoinsieme di dati relativo al periodo 2019–2020.

In [ ]:
for feature in features_to_plot:
    data_slice = slice_groups['Group by Day'][slice_groups['Group by Day'].columns.intersection(pollutants[feature])]
    data_slice.query('datetime > 2019 and datetime < 2020').plot(title=f'{feature} in year 2019-2020', figsize=(12,4)).set(xlabel=None)

Si osserva un incremento nei valori di `Particulate Matter`, `Nitrogen Compounds` e `Carbon Monoxide` a partire da ottobre, con un picco che tende a persistere fino circa a marzo. Al contrario, la `Ozone Concentration` mostra un comportamento opposto, raggiungendo i valori massimi indicativamente tra maggio e giugno.

### PairPlot
Andiamo ad utilizzare il grafico a coppie, che ci consente di visualizzare in modo più chiaro le relazioni bivariate tra le variabili, nonché la distribuzione univariata di ciascuna di esse.

In [ ]:
sns.pairplot(slice_groups['Group by Month'])

È evidente una correlazione lineare significativa tra `NOx`, `NO` e `NO2`. Considerando questa relazione, può essere opportuno mantenere esclusivamente la variabile aggregata `NOx` come rappresentazione generale del gruppo.

### Matrice di correlazione
Ora, andiamo ad utilizzare la matrice di correlazione che offre una rappresentazione sintetica ed efficace del grado di associazione lineare tra le diverse variabili del dataset.

In [ ]:
corr = slice_groups['Group by Day'].corr(numeric_only=True).round(2)
mask = np.triu(np.ones_like(corr, dtype=bool))

plt.figure(figsize=(10,5))
sns.heatmap(data=corr, mask=mask, annot=True, cmap="rocket_r")
plt.show()

In [ ]:
corr_target = abs(corr['PM2.5 (ug/m3)'])
relevant_features = corr_target[corr_target>0.4]
relevant_features.sort_values(ascending=False)
df_india.head()

Il grafico evidenzia diverse correlazioni significative tra le variabili. In particolare:

- `NOx` mostra una forte correlazione con le variabili `NO` e `NO2`.
- È inoltre evidente una relazione positiva tra `PM2.5` e `NOx`, suggerendo che all’aumentare dei valori di `NOx`, tendono ad aumentare anche i livelli di `PM2.5`.

## Feature Engineering

### Eliminazione delle Feature correlate

In [ ]:
df_india = df_india.drop(['NO (ug/m3)', 'NO2 (ug/m3)'], axis=1)
df_india.head()

### Resampling
Poiché il dataframe combinato include misurazioni provenienti da diverse località all'interno dello stesso stato e riferite agli stessi intervalli temporali, è possibile che si verifichino duplicazioni temporali. Dal momento che l’obiettivo è analizzare la qualità dell’aria a livello statale, procederemo con un ricampionamento temporale aggregando i dati mediante media delle misurazioni corrispondenti allo stesso timestamp.

In [ ]:
df_resampled = (
    df_india
    .groupby('state')  # Resample within each state
    .resample('60min')
    .mean(numeric_only=True)
    .reset_index()
)
df_resampled = df_resampled.set_index('datetime')
df_india=df_resampled.copy()

### Isolation Forest - Rilevamento e Rimozione degli Outlier
In questa sequenza di celle utilizzeremo l'algoritmo Isolation Forest per identificare e rimuovere gli outlier, che rappresentano valori anomali che si discostano in modo significativo dalla distribuzione generale dei dati. La loro presenza può compromettere l’accuratezza delle analisi statistiche e influenzare negativamente le prestazioni dei modelli predittivi. L’identificazione e la rimozione degli outlier consente di ottenere risultati più affidabili e modelli previsionali più robusti.

Definiamo le colonne su cui vogliamo applicare l'Isolation Forest.

In [ ]:
features = ['PM2.5 (ug/m3)', 'CO (mg/m3)', 'Ozone (ug/m3)', 'NOx (ug/m3)']
df_india_features = df_india[features].copy()

Creiamo il modello specificando la proporzione di outlier attesi (`contamination`).

In [ ]:
iso = IsolationForest(contamination=0.01, random_state=42)
iso.fit(df_india_features)

Usiamo il metodo `predict` per assegnare -1 agli outlier e 1 ai punti normali.

In [ ]:
df_india['anomaly'] = iso.predict(df_india_features)

Creiamo un nuovo DataFrame senza gli outlier identificati.

In [ ]:
df_india_clean = df_india[df_india['anomaly'] == 1].drop(columns='anomaly')

Confrontiamo la distribuzione originale e quella ripulita per ogni feature.

In [ ]:
fig, axes = plt.subplots(4, 2, figsize=(14, 12))
for i, col in enumerate(features):
    # distribuzione originale
    axes[i, 0].hist(df_india[col].dropna(), bins=100)
    axes[i, 0].set_title(f"Originale: {col}")
    # distribuzione pulita
    axes[i, 1].hist(df_india_clean[col].dropna(), bins=100)
    axes[i, 1].set_title(f"Pulita: {col}")
plt.tight_layout()
plt.show()

df_india = df_india_clean.copy()
df_india.head()

### Gestione dei valori mancanti

In [ ]:
get_null_info(df_india)

In [ ]:
numeric_cols = df_india.select_dtypes(include='number').columns

df_india[numeric_cols] = df_india[numeric_cols].interpolate(method='pad')
df_india[numeric_cols] = df_india[numeric_cols].fillna(df_india[numeric_cols].mean())
df_india.info()

### Arricchimento del Dataset con Caratteristiche Aggiuntive
Procediamo con l'ampliamento del nostro dataset, integrando nuove features che possano risultare utili.

In [ ]:
def create_features(df):
    df = df.copy()
    df['hour']       = df.index.hour
    df['dayofmonth'] = df.index.day
    df['dayofweek']  = df.index.dayofweek
    df['dayofyear']  = df.index.dayofyear
    df['weekofyear'] = df.index.isocalendar().week.astype("int64")
    df['month']      = df.index.month
    df['quarter']    = df.index.quarter
    df['year']       = df.index.year
    return df

In [ ]:
date_features = ['hour', 'dayofmonth', 'dayofweek', 'dayofyear', 'weekofyear', 'month', 'quarter', 'year']
df_india = create_features(df_india)

In [ ]:
pd.set_option('display.max_columns', None)
df_india.head()

Ora, grazie alle features precedentemente descritte, è semplice visualizzare le diverse metriche. Ad esempio, possiamo esaminare la qualità dell'aria nel corso dei mesi utilizzando un boxplot.

In [ ]:
def plot_by_datetime(metric, time_groups):
    for time_group in time_groups:
        fig, ax = plt.subplots(figsize=(12, 4))
        sns.boxplot(data=df_india, x=time_group, y=metric, hue=time_group, palette="icefire", showfliers=False, legend=False)
        ax.set_title(f'{metric} by {time_group}')
        ax.set(xlabel=time_group)
        plt.show()

In [ ]:
plot_by_datetime('PM2.5 (ug/m3)', ['hour', 'dayofmonth', 'dayofweek', 'weekofyear', 'month', 'quarter', 'year'])

I grafici mostrano chiaramente che i vari gruppi di date catturano tendenze e informazioni significative. Un punto interessante è che il vettore di feature `dayofweek` potrebbe non essere così rilevante, dato che la distribuzione appare simile per tutti i giorni della settimana. Tuttavia, includeremo comunque tutte queste informazioni nel nostro modello.

# Dataset sulla **Qualità dell'Aria in Cina**



Il dataset in questione è stato messo a disposizione dal Beijing Municipal Environmental Monitoring Center, l’ente ufficiale del Governo Cinese deputato al monitoraggio e alla gestione dell’inquinamento atmosferico, al fine di raccogliere informazioni relative alle condizioni della qualità dell’aria nel distretto di Beijing nel periodo compreso tra il 2013 e il 2017.

Sempre citando la documentazione ufficiale, il dataset permette di indagare su numerose variabili ambientali e parametri atmosferici che includono:

- PM10 e PM2.5: concentrazioni di particolato in ug/m³;
- CO: monossido di carbonio (mg/m³);
- NO₂: biossido di azoto (ug/m³);
- SO₂: anidride solforosa (ug/m³);
- O3: concentrazione di Ozono (ug/m^3);
- Parametri meteorologici e ambientali quali temperatura, pressione barometrica, temperatura del punto di rugiada, precipitazioni, velocità e direzione del vento.

Unione dei vari CSV in un unico file

In [ ]:
csv_folder = china_extract_to
output_file = os.path.join(china_extract_to, 'combined_dataset.csv')


csv_files = [f for f in os.listdir(csv_folder) if f.endswith('.csv')]
print(f"Found {len(csv_files)} CSV files.")


dfs = []
for file in csv_files:
    path = os.path.join(csv_folder, file)
    try:
        df_china = pd.read_csv(path)
        dfs.append(df_china)
        print(f"Loaded {file} with {len(df_china)} rows.")
    except Exception as e:
        print(f"Error reading {file}: {e}")

if not dfs:
    raise ValueError("No CSV files were successfully loaded.")

combined_df = pd.concat(dfs, ignore_index=True)
print(f"\nCombined DataFrame has {len(combined_df)} rows and {len(combined_df.columns)} columns.")


combined_df.to_csv(output_file, index=False)
print(f"\nCombined CSV saved to: {output_file}")

Si procede alla lettura del file contenente le informazioni relative alle varie stazioni di monitoraggio. Successivamente, verranno rimosse alcune colonne ritenute non necessarie per l'analisi in corso, al fine di semplificare la struttura del dataset ed enfatizzare solo le informazioni rilevanti.

In [ ]:
dataframe = pd.read_csv(f'{china_extract_to}/combined_dataset.csv')
dataframe.head()

In [ ]:
dataframe.drop(columns=['No'], inplace=True)
dataframe.head()

Si crea una lista di tutti i distretti presenti nel dataset, assicurandosi di includere ciascun nome una sola volta. Questo passaggio fornisce una visione d’insieme delle regioni coperte dai dati, supportando analisi geografiche e suddivisioni successive.

In [ ]:
unique_cities = dataframe['station'].unique()
unique_cities

Successivamente si crea una vista del numero di entry per ogni distretto. Questo passaggio fornisce una visone dell'uniformità del quantitativo dei dati presenti in ogni distretto.

In [ ]:
# Quick overview
print(f"[{dataframe['station'].nunique()}] different cities and [{dataframe['station'].count()}] total records available.")

# Get city counts
cities = dataframe["station"].value_counts()

cities.plot.pie(
    labels=[f"{c}: {p} records" for c, p in zip(cities.index, cities.values)],
    autopct="%.1f%%",
    shadow=True,
    figsize=(7,7),
    title="Cities and Record Counts"
);
plt.ylabel('');
plt.show()

Unione delle colonne temporali

In [ ]:
dataframe['datetime'] = pd.to_datetime(
    dataframe[['year', 'month', 'day', 'hour']],
    errors='coerce'
)

# (Optional) Drop the original columns if you no longer need them
dataframe.drop(columns=['year', 'month', 'day', 'hour'], inplace=True)

# Preview the result
dataframe.head()

## Pre-elaborazione dei dati

#### Utilizzo di ‘datetime’ come indice temporale

Per gestire efficacemente le serie storiche, viene utilizzata la colonna `datetime` come indice datetime.

In [ ]:
dataframe = dataframe.rename(columns={'station': 'state'})
dataframe = dataframe.set_index('datetime')
dataframe.head()

### Verifica dei valori mancanti

Il primo passo consiste nel quantificare quanti dati mancanti siano presenti per ciascuna delle feature selezionate.

In [ ]:
dataframe.isnull().sum().sort_values(ascending=False)


In [ ]:
df_china=dataframe
df_china = df_china.dropna(how='all')
df_china = df_china.dropna(how='all', axis='columns')


In [ ]:
def get_null_info(dataframe):
    null_vals = dataframe.isnull().sum()

    df_null_vals = pd.concat({'Null Count': null_vals,
                              'Percentage of Missing Values (%)': round(null_vals * 100 / len(dataframe), 2)}, axis=1)

    return df_null_vals.sort_values(by=['Null Count'], ascending=False)

In [ ]:
df_null_info = get_null_info(df_china)

plt.figure(figsize=(8, 10))
sns.barplot(data=df_null_info, x='Percentage of Missing Values (%)', y=df_null_info.index, orient='h', color='steelblue')
plt.show()

### Eliminare i valori mancanti per soglia

Non essendoci colonne al di sopra di una certa soglia (>40%) non viene eliminato nulla.

### Analisi esplorativa dei dati

Sto raccogliendo le metriche iniziali in diversi gruppi. Ciò consentirà di effettuare confronti migliori.

In [ ]:
pollutants = {
    'Particulate Matter' : ['PM2.5', 'PM10'],
    'Nitrogen Compounds' : ['NO2'],
    'Carbon Monoxide': ['CO'],
    'Sulfur Dioxide': ['SO2'],
    'Ozone Concentration' : ['O3']
}
other_metrics = {
    'Pressure' : ['PRES'],
    'Temperatures' : ['TEMP'],
    'Dew Point Temperature' : ['DEWP'],
    'Rainfall' : ['RAIN'],
    'Wind Direction' : ['wd'],
    'Wind Speed' : ['WSPM']
}

### Frequenze temporali

Cominciamo a raggruppare il nostro DataFrame per varie frequenze temporali.

In [ ]:
slice_groups = {
    'Group by Day':   df_china.groupby(pd.Grouper(freq='1D')).mean(numeric_only=True),
    'Group by Month': df_china.groupby(pd.Grouper(freq='1ME')).mean(numeric_only=True),
    'Group by Year':  df_china.groupby(pd.Grouper(freq='1YE')).mean(numeric_only=True)
}

In [ ]:
def plot_features_by_group(features, slice_groups):
    for feature in features:
        fig, ax = plt.subplots(1, 1, figsize=(12, 4))
        fig.suptitle(feature)

        labels = []
        for i, (group, group_df) in enumerate(slice_groups.items()):
            data_slice = group_df[group_df.columns.intersection(pollutants[feature])]



            data_slice.plot(kind="line", ax=ax)

            for column in data_slice.columns:
                labels.append(f'{column} [{group}]')

        ax.set(xlabel=None)
        ax.legend(labels)
        plt.plot()

In [ ]:
features_to_plot = ['Particulate Matter', 'Carbon Monoxide', 'Ozone Concentration', 'Nitrogen Compounds']
plot_features_by_group(features_to_plot, slice_groups)


### Analisi stagionale su base annua
Dalle metriche selezionate emergono potenziali pattern di tipo stagionale. Per approfondire questa osservazione, eseguiamo un’analisi dettagliata delle variazioni stagionali nell’arco di un anno. Come punto di partenza, prenderemo in considerazione un sottoinsieme di dati relativo al periodo 2016–2017.

In [ ]:
for feature in features_to_plot:
    data_slice = slice_groups['Group by Day'][slice_groups['Group by Day'].columns.intersection(pollutants[feature])]
    data_slice.query('datetime > 2016 and datetime < 2017').plot(title=f'{feature} in year 2016-2017', figsize=(12,4)).set(xlabel=None)

Si osserva un incremento nei valori di `Particulate Matter`, `Nitrogen Compounds` e `Carbon Monoxide` a partire da ottobre, con un picco che tende a persistere fino circa a marzo. Al contrario, la `Ozone Concentration` mostra un comportamento opposto, raggiungendo i valori massimi indicativamente tra maggio e giugno.

### PairPlot
Andiamo ad utilizzare il grafico a coppie, che ci consente di visualizzare in modo più chiaro le relazioni bivariate tra le variabili, nonché la distribuzione univariata di ciascuna di esse.

In [ ]:
sns.pairplot(slice_groups['Group by Month'])

### Matrice di correlazione
Ora, andiamo ad utilizzare la matrice di correlazione che offre una rappresentazione sintetica ed efficace del grado di associazione lineare tra le diverse variabili del dataset.

In [ ]:
corr = slice_groups['Group by Day'].corr(numeric_only=True).round(2)
mask = np.triu(np.ones_like(corr, dtype=bool))

plt.figure(figsize=(10,5))
sns.heatmap(data=corr, mask=mask, annot=True, cmap="rocket_r")
plt.show()

In [ ]:
corr_target = abs(corr['PM2.5'])
relevant_features = corr_target[corr_target>0.4]
relevant_features.sort_values(ascending=False)

Il grafico evidenzia diverse correlazioni significative tra le variabili.

Vengono raggruppare le direzioni cardinali del vento mostrando la loro influenza sulla concentrazione di PM2.5

In [ ]:
# Apply to the dataframe
def wind_quadrant_str(direction):
    if pd.isna(direction):
        return "Unknown"
    direction = direction.upper()

    if direction in ['N', 'NNE', 'NNW']:
        return "N"
    elif direction in ['NE']:
        return "NE"
    elif direction in ['E', 'ENE', 'ESE']:
        return "E"
    elif direction in ['NW']:
        return "NW"
    elif direction in ['S', 'SSE', 'SSW']:
        return "S"
    elif direction in ['SE']:
        return "SE"
    elif direction in ['W', 'WNW', 'WSW']:
        return "W"
    elif direction in ['SW']:
        return "SW"
    else:
        return "Other"

# Apply to the dataframe
dataframe["wind_quadrant"] = dataframe["wd"].apply(wind_quadrant_str)

# Boxplot
ax = dataframe.boxplot(
    column="PM2.5",
    by="wind_quadrant",
    showmeans=True,
    grid=False
)
ax.set_ylabel("PM2.5 Concentration (µg/m³)")
ax.set_title("")
plt.suptitle("")
plt.ylim(0, 150)
plt.show()


### Resampling
Poiché il dataframe combinato include misurazioni provenienti da diverse località all'interno dello stesso stato e riferite agli stessi intervalli temporali, è possibile che si verifichino duplicazioni temporali. Dal momento che l’obiettivo è analizzare la qualità dell’aria a livello statale, procederemo con un ricampionamento temporale aggregando i dati mediante media delle misurazioni corrispondenti allo stesso timestamp.

In [ ]:
df_resampled = (
    df_china
    .groupby('state')  # Resample within each state
    .resample('60min')
    .mean(numeric_only=True)
    .reset_index()
)
df_resampled = df_resampled.set_index('datetime')
df_china=df_resampled.copy()

### Isolation Forest - Rilevamento e Rimozione degli Outlier
Utilizzeremo l'algoritmo Isolation Forest per identificare e rimuovere automaticamente gli outlier dalle nostre quattro variabili ambientali.

Definiamo le colonne su cui applicheremo Isolation Forest: `PM2.5`, `CO`, `O3` e `NO2`.

In [ ]:
features = ['PM2.5', 'CO', 'O3', 'NO2']
df_china_features = df_china[features].copy()

Impostiamo il parametro `contamination` in base alla percentuale di outlier attesa (qui 1%).

In [ ]:
iso = IsolationForest(contamination=0.01, random_state=42)
iso.fit(df_china_features)

Con `predict`, i valori anomali vengono etichettati con -1, quelli normali con +1.

In [ ]:
df_china['anomaly'] = iso.predict(df_china_features)

Creiamo un nuovo DataFrame `df_clean` escludendo tutte le righe etichettate come outlier.

In [ ]:
df_china_clean = df_china[df_china['anomaly'] == 1].drop(columns='anomaly')

Confrontiamo le distribuzioni originali e quelle ripulite per ciascuna variabile.

In [ ]:
fig, axes = plt.subplots(4, 2, figsize=(14, 12))
for i, col in enumerate(features):
    # istogramma dati originali
    axes[i, 0].hist(df_china[col].dropna(), bins=100)
    axes[i, 0].set_title(f"Originale: {col}")
    # istogramma dati puliti
    axes[i, 1].hist(df_china_clean[col].dropna(), bins=100)
    axes[i, 1].set_title(f"Pulita: {col}")
plt.tight_layout()
plt.show()

df_china = df_china_clean.copy()
df_china.head()

### Gestione dei valori mancanti

In [ ]:
get_null_info(df_china)
df_china.head()

In [ ]:
numeric_cols = df_china.select_dtypes(include='number').columns

df_china[numeric_cols] = df_china[numeric_cols].interpolate(method='pad')
df_china[numeric_cols] = df_china[numeric_cols].fillna(df_china[numeric_cols].mean())

df_china.info()

### Arricchimento del Dataset con Caratteristiche Aggiuntive
Procediamo con l'ampliamento del nostro dataset, integrando nuove features che possano risultare utili.

In [ ]:
def create_features(df):
    df = df.copy()
    df['hour']       = df.index.hour
    df['dayofmonth'] = df.index.day
    df['dayofweek']  = df.index.dayofweek
    df['dayofyear']  = df.index.dayofyear
    df['weekofyear'] = df.index.isocalendar().week.astype("int64")
    df['month']      = df.index.month
    df['quarter']    = df.index.quarter
    df['year']       = df.index.year
    return df

In [ ]:
date_features = ['hour', 'dayofmonth', 'dayofweek', 'dayofyear', 'weekofyear', 'month', 'quarter', 'year']
df_china = create_features(df_china)

Ora, grazie alle features precedentemente descritte, è semplice visualizzare le diverse metriche. Ad esempio, possiamo esaminare la qualità dell'aria nel corso dei mesi utilizzando un boxplot.

In [ ]:
def plot_by_datetime(metric, time_groups):
    for time_group in time_groups:
        fig, ax = plt.subplots(figsize=(12, 4))
        sns.boxplot(data=df_china, x=time_group, y=metric, hue=time_group, palette="icefire", showfliers=False, legend=False)
        ax.set_title(f'{metric} by {time_group}')
        ax.set(xlabel=time_group)
        plt.show()

In [ ]:
plot_by_datetime('PM2.5', ['hour', 'dayofmonth', 'dayofweek', 'weekofyear', 'month', 'quarter', 'year'])
df_china.head()

I grafici mostrano chiaramente che i vari gruppi di date catturano tendenze e informazioni significative. Un punto interessante è che il vettore di feature `dayofweek` potrebbe non essere così rilevante, dato che la distribuzione appare simile per tutti i giorni della settimana. Tuttavia, includeremo comunque tutte queste informazioni nel nostro modello.

# Augmented Data

Il dataset in questione è stato generato da diversi LLM, tra cui: Deepseek (500 entries), Gemini (3900 entries), Claude (300 entries), Qwen (500 entries), Mistral (500 entries), Meta (400 entries), Grock (100 entries), ChatGpt (200 entries), Hunyuan (350 entries), Tencent (400 entries).
Il numero totali di entries utilizzate è proporzionale alla capacità del modello (versione gratuita/prova)  di generare grandi quantitativi di dati.
Il dataset è stato generato prendendo come base i dati dell'unione dei dataset precedenti e performando data augmentation con i modelli menzionati in precedenza.
Il datasef fa riferimento all'anno 2021 e mantiene le stesse variabili ambientali e parametri atmosferici dei precedenti dataset, rendendolo già omogeneo.
I quali includono:

- PM10 e PM2.5: concentrazioni di particolato in ug/m³;
- CO e CO₂: rispettivamente monossido e anidride carbonica, misurati in vari formati (mg/m³, ppm, ecc.);
- NO, NO₂ e NOx: varianti degli ossidi di azoto, riportati in unità adatte (ug/m³, ppb, ppm);
- SO₂, NH₃ e altri inquinanti quali Benzene, CH₄, e composti organici come MP-Xylene, Eth-Benzene, O Xylene, e Xylene;
- Parametri meteorologici e ambientali quali temperatura, pressione barometrica, umidità relativa, velocità e direzione del vento, radiazione solare e precipitazioni.


Caricamento del dataset

In [ ]:
csv_folder = "datasets/"
df_augmented = pd.read_csv(f'{csv_folder}/Augmented_data.csv')
df_augmented.head()

Verifico che gli stati generati appaiano una sola volta all'interno del dataset

In [ ]:
valid_states = df_augmented['state'].unique()
valid_states

Verifico il numero di entries per stato

In [ ]:
# Quick overview
print(f"[{df_augmented['state'].nunique()}] different cities and [{df_augmented['state'].count()}] total records available.")

# Get city counts
cities = df_augmented["state"].value_counts()

cities.plot.pie(
    labels=[f"{c}: {p} records" for c, p in zip(cities.index, cities.values)],
    autopct="%.1f%%",
    shadow=True,
    figsize=(7,7),
    title="States and Record Counts"
);
plt.ylabel('');
plt.show()

Si nota che i modelli, durante la loro generazione, hanno generato meno stati rispetto ai 32 del dataset omogeneizzato
Il quantitativo dei valori generati ammonta a circa 6900 entries.

In [ ]:
def plot_by_datetime(metric, time_groups):
    for time_group in time_groups:
        fig, ax = plt.subplots(figsize=(12, 4))
        sns.boxplot(data=df_augmented, x=time_group, y=metric, hue=time_group, palette="icefire", showfliers=False, legend=False)
        ax.set_title(f'{metric} by {time_group}')
        ax.set(xlabel=time_group)
        plt.show()

In [ ]:
plot_by_datetime('PM2.5', ['dayofmonth', 'dayofweek', 'weekofyear', 'month', 'quarter', 'year'])
df_china.head()

# Omogeneizzazione ed Unione dei DataFrame `df_india` e `df_china`

In questa sezione andremo a uniformare ed unire la struttura dei due DataFrame, contenenti dati sulla qualità dell'aria dell'India e della Cina, in un unico DataFrame per l'allenamento dei modelli di Regressione.


## Rinomina delle colonne in `df_india` per uniformarle a quelle di `df_china`

In questa cella rinominiamo alcune colonne di `df_india` in modo che abbiano gli stessi nomi utilizzati in `df_china`.
Ad esempio:
- `"PM2.5 (ug/m3)"` diventerà `"PM2.5"`
- `"CO (mg/m3)"` diventerà `"CO"`
- `"Ozone (ug/m3)"` diventerà `"O3"`


In [ ]:
rename_map_df = {
    'PM2.5 (ug/m3)': 'PM2.5',
    'CO (mg/m3)': 'CO',
    'Ozone (ug/m3)': 'O3',
    'NOx (ug/m3)': 'NOx'
}

df_india.rename(columns=rename_map_df, inplace=True)

## Estrazione delle colonne comuni tra `df_india` e `df_china`

Una volta uniformati i nomi delle colonne, cerchiamo l'intersezione tra le colonne dei due DataFrame, cioè quelle che sono presenti in entrambi.
Questo ci permetterà di creare due DataFrame omogenei e pronti per essere concatenati.


In [ ]:
common_columns = list(set(df_india.columns) & set(df_china.columns))

## Creazione dei DataFrame `df_india_aligned` e `df_china_aligned`

In questa fase creiamo due nuovi DataFrame (`df_aligned` e `df_china_aligned`) che contengono solo le colonne comuni.
In questo modo abbiamo una base uniforme su cui allenare i modelli in seguito.

In [ ]:
df_aligned = df_india[common_columns].copy()
df_china_aligned = df_china[common_columns].copy()

print("Colonne comuni (omogeneizzate):")
print(df_aligned.columns)

## Concatenazione dei DataFrame omogeneizzati

Dopo aver uniformato le colonne, possiamo unire i due DataFrame in un unico dataset più grande. Questo ci permette di avere tutti i dati in un'unica struttura per analisi successive. Impostiamo il parametro `ignore_index=True` per riordinare gli indici nel DataFrame concatenato.

In [ ]:
df = pd.concat([df_aligned, df_china_aligned], ignore_index=True)

## Verifica del risultato

Stampiamo le dimensioni e alcune righe del nuovo DataFrame concatenato `df` per assicurarci che l’unione sia avvenuta correttamente.\


In [ ]:
print("Numero di righe:", df.shape[0])
print("Numero di colonne:", df.shape[1])

df.sample(5)

## Riduzione Sampling

Riduciamo il timeframe compreso all'interno del Dataframe al periodo temporale in comune per entrambi i dataset.


In [ ]:
df = df[(df['year'] >= 2015) & (df['year'] <= 2022)]
df = df.reset_index(drop=True)
df.head()

In [ ]:
# 1. Drop the 'hour' column
df = df.drop(columns=['hour'])

# 2. Group by date and state, and average the pollution values
df_daily = (
    df.groupby(['year', 'month', 'dayofmonth', 'state'], as_index=False)
      .agg({
          'PM2.5': 'mean',
          'CO': 'mean',
          'O3': 'mean',
          'dayofweek': 'first',
          'quarter': 'first',
          'weekofyear': 'first',
          'dayofyear': 'first'
      })
)

df_daily.head()

## Integrazione Augmented Dataset

Inseriamo all'interno del dataframe omogeneizzato i valori generati dai vari llm sostituendo l'anno 2021


In [ ]:
# Split the original df
before_2021_unf = df_daily[df_daily['year'] < 2021]
after_2021 = df_daily[df_daily['year'] > 2021]

before_2021 = before_2021_unf[before_2021_unf['state'].isin(valid_states)].copy()

# Concatenate in order: before -> new 2021 -> after
df = pd.concat([before_2021, df_augmented, after_2021], ignore_index=True)

df.head()


# Aggiunta Lag Features

Le cosiddette “lag features” consentono di includere nei modelli i valori storici di una variabile, risultando spesso determinanti nelle previsioni grazie al loro elevato potere predittivo. Possiamo inoltre generare lag anche per altre variabili significative, ampliando il contesto informativo del dataset e potenzialmente migliorando la precisione delle stime.

Analizzando i boxplot, abbiamo osservato che alcune feature evidenziano trend stagionali o andamenti rilevanti nel tempo. Sulla base di queste evidenze, creeremo lag features mirate per sfruttare al meglio tali pattern.

In [ ]:
def create_lag_features(df):
    df = df.copy()
    df['pm_lag_1Y'] = df['PM2.5'].shift(365)   # 1 year lag
    df['pm_lag_2Y'] = df['PM2.5'].shift(730)   # 2 year lag
    return df

In [ ]:
lag_features = ['pm_lag_1Y', 'pm_lag_2Y']
df = create_lag_features(df)
df_daily = create_lag_features(df_daily)
df.head()
df.info()

A seguito della creazione delle lag features, riscontriamo che i primi record del dataset presentano valori mancanti: ciò è inevitabile, dato che non esistono dati storici precedenti per calcolare i ritardi temporali. È quindi fondamentale gestire con cura questi missing values, poiché molti algoritmi predittivi non possono elaborare dati incompleti.

# Creazione Dataset per l'allenamento

Esportiamo i due dataset, originale e augmented, per proseguire con l'allenamento nel file successivo


In [ ]:
df.to_csv('./original_dataset.csv', sep = ',', index = False)
df_daily.to_csv('./augmented_dataset.csv', sep = ',', index = False)

In [ ]:
if running_in_colab:
  google.colab.files.download('./original_dataset.csv')
  google.colab.files.download('./augmented_dataset.csv')